# Run the CICI SAFE vSDX Using Chameleon

#### Configure the Envrionment

In [14]:
# Set up user's project (user's can be multiple ones, so there is no default currently)
export OS_PROJECT_NAME='CH-816532'

# Set region (again, no default currently)
export OS_REGION_NAME='CHI@TACC'

# Set chameleon keypair name and path to the private ssh key
export SSH_KEY_NAME=${USERNAME}'-jupyter'
export SSH_PRIVATE_KEY=${HOME}/.ssh/id_rsa

# Set the reservations to use.  
# CONTROLLER_RESERVATION_ID can be for any type of node
export SDX_RESERVATION_ID='8478da29-7d05-4a65-a02a-0bc73557e196'
export RIAK_RESERVATION_ID='8478da29-7d05-4a65-a02a-0bc73557e196'

#Set the working directory
export WORKING_DIR=${HOME}/work/CICI-SAFE/Chameleon-Jupyter/SDX

#Set GENI CERT file
export GENI_CERT=${HOME}/work/geni-pruth.pem

#### Start RIAK Server

In [15]:
cd $WORKING_DIR

RIAK_NETWORK='sharednet1'
RIAK_NODE_NAME=${USERNAME}-riak
RIAK_STACK_NAME=${USERNAME}-riak

In [16]:
echo Creating RIAK server. This will take several minutes!
openstack stack create  --max-width 80 \
        --template riak.yaml \
        --parameter "key_name=${SSH_KEY_NAME}" \
        --parameter "reservation_id=${RIAK_RESERVATION_ID}" \
        --parameter "network_name=${RIAK_NETWORK}" \
        --parameter "riak_node_name=${RIAK_NODE_NAME}" \
        --wait \
        ${RIAK_STACK_NAME}

Creating RIAK server. This will take several minutes!
2019-02-26 18:41:04Z [pruth-riak]: CREATE_IN_PROGRESS  Stack CREATE started
2019-02-26 18:41:05Z [pruth-riak.riak_floating_ip]: CREATE_IN_PROGRESS  state changed
2019-02-26 18:41:07Z [pruth-riak.riak-server]: CREATE_IN_PROGRESS  state changed
2019-02-26 18:41:08Z [pruth-riak.riak_floating_ip]: CREATE_COMPLETE  state changed
2019-02-26 18:50:20Z [pruth-riak.riak-server]: CREATE_COMPLETE  state changed
2019-02-26 18:50:21Z [pruth-riak.riak_ip_association]: CREATE_IN_PROGRESS  state changed
2019-02-26 18:50:24Z [pruth-riak.riak_ip_association]: CREATE_COMPLETE  state changed
2019-02-26 18:50:24Z [pruth-riak]: CREATE_COMPLETE  Stack CREATE completed successfully
+---------------------+--------------------------------------------------------+
| Field               | Value                                                  |
+---------------------+--------------------------------------------------------+
| id                  | 96935b20-45a

In [17]:
echo RIAK server creation complete! 
echo ${RIAK_NODE_NAME} status `openstack server show  --format value -c status ${RIAK_NODE_NAME}`

RIAK_PUBLIC_IP=`openstack server show  --format value -c addresses ${RIAK_NODE_NAME} | cut -d " " -f 2`
echo The RIAK se public IP is $RIAK_PUBLIC_IP

RIAK server creation complete!
pruth-riak status ACTIVE
The RIAK se public IP is 129.114.108.53


#### Start SDX Server

In [18]:
cd $WORKING_DIR

SDX_NETWORK='sharednet1'
SDX_NODE_NAME=${USERNAME}-sdx
SDX_STACK_NAME=${USERNAME}-sdx

In [19]:
echo Creating SDX server. This will take several minutes!
openstack stack create  --max-width 80 \
        --template sdx.yaml \
        --parameter "key_name=${SSH_KEY_NAME}" \
        --parameter "reservation_id=${SDX_RESERVATION_ID}" \
        --parameter "network_name=${SDX_NETWORK}" \
        --parameter "sdx_node_name=${SDX_NODE_NAME}" \
        --parameter "riak_ip=${RIAK_PUBLIC_IP}" \
        --wait \
        ${SDX_STACK_NAME}

Creating SDX server. This will take several minutes!
2019-02-26 18:53:29Z [pruth-sdx]: CREATE_IN_PROGRESS  Stack CREATE started
2019-02-26 18:53:30Z [pruth-sdx.sdx_floating_ip]: CREATE_IN_PROGRESS  state changed
2019-02-26 18:53:32Z [pruth-sdx.sdx_floating_ip]: CREATE_COMPLETE  state changed
2019-02-26 18:53:34Z [pruth-sdx.sdx]: CREATE_IN_PROGRESS  state changed
2019-02-26 19:04:33Z [pruth-sdx.sdx]: CREATE_COMPLETE  state changed
2019-02-26 19:04:33Z [pruth-sdx.sdx_ip_association]: CREATE_IN_PROGRESS  state changed
2019-02-26 19:04:36Z [pruth-sdx.sdx_ip_association]: CREATE_COMPLETE  state changed
2019-02-26 19:04:36Z [pruth-sdx]: CREATE_COMPLETE  Stack CREATE completed successfully
+---------------------+--------------------------------------------------------+
| Field               | Value                                                  |
+---------------------+--------------------------------------------------------+
| id                  | be935e78-81c6-4973-89c3-0540471d343a     

In [21]:
echo SDX server creation complete! 
echo ${SDX_NODE_NAME} status `openstack server show  --format value -c status ${SDX_NODE_NAME}`

SDX_PUBLIC_IP=`openstack server show  --format value -c addresses ${SDX_NODE_NAME} | cut -d " " -f 2`
echo The SDX se public IP is $SDX_PUBLIC_IP

SDX server creation complete!
pruth-sdx status ACTIVE
The SDX se public IP is 129.114.108.17


In [22]:
SDX_PUBLIC_IP=`openstack server show --format value -c addresses ${SDX_NODE_NAME} | cut -d " " -f 2`
echo SDX_PUBLIC_IP is $SDX_PUBLIC_IP

#Poll for node to be ready for ssh
while [[ true ]]
do 
	ssh -q -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${SDX_PUBLIC_IP} \
    exit
    
    if [[ "$?" == "0" ]]; then break; fi
    
    echo Node not ready for ssh. retrying in 5 seconds.
    sleep 5
done

echo SDX node ready for ssh

SDX_PUBLIC_IP is 129.114.108.17
SDX node ready for ssh


In [23]:
#Tail log until complete. Note: YOU MUST MANUALLY STOP THIS TO CONTINUE!
ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${SDX_PUBLIC_IP} \
    sudo tail /root/boot.log

[INFO] Installing artifact /root/CICI-SAFE/SDX-Simple/SAFE_SDX/target/exoplex.sdx-simple-0.1-SNAPSHOT.jar to /root/CICI-SAFE/SDX-Simple/SAFE_SDX/target/appassembler/repo/safe/exoplex.sdx-simple/0.1-SNAPSHOT/exoplex.sdx-simple-0.1-SNAPSHOT.jar
[INFO] ------------------------------------------------------------------------
[INFO] BUILD SUCCESS
[INFO] ------------------------------------------------------------------------
[INFO] Total time: 1:20.411s
[INFO] Finished at: Tue Feb 26 19:23:23 UTC 2019
[INFO] Final Memory: 33M/1100M
[INFO] ------------------------------------------------------------------------
Create ssh key
Boot Script Done!


#### Configure the SDX node

In [24]:
#Copy GENI pem file to node
REMOTE_GENI_CERT=/home/cc/geni.pem

echo Copying GENI cert to cc@$SDX_PUBLIC_IP:~/geni.pem
scp -q -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    ${GENI_CERT} \
    cc@${SDX_PUBLIC_IP}:${REMOTE_GENI_CERT}
echo Done

Copying GENI cert to cc@129.114.108.17:~/geni.pem
Done


In [25]:
LOCAL_SDX_CONFIG=${WORKING_DIR}/sdx.conf

REMOTE_SDX_CONFIG=/home/cc/sdx.conf
REMOTE_WORKING_DIR=/root
REMOTE_BIN_DIR=${REMOTE_WORKING_DIR}/CICI-SAFE/SDX-Simple/SAFE_SDX/target/appassembler/bin

DATAPLANE_SLICE_NAME=${USERNAME}-sdx-dataplane

cat << EOF > $LOCAL_SDX_CONFIG
config {
safe=false
slicename="${DATAPLANE_SLICE_NAME}"
type="server"
sshkey="/root/.ssh/id_rsa"
safekey="bphJZn3RJBnNqoCZk6k9SBD8mwSb054PXbwV7HpE80E"
exogenipem="${REMOTE_GENI_CERT}"
exogenism="https://geni.renci.org:11443/orca/xmlrpc"
ipprefix="192.168.30.1/20"
riakserver="${RIAK_PUBLIC_IP}"
serverinslice=false
safeserver="${SDX_PUBLIC_IP}"
plexusserver="${SDX_PUBLIC_IP}"
serverurl="http://0.0.0.0:8888/"
scriptsdir="${REMOTE_WORKING_DIR}/CICI-SAFE/SDX-Simple/SAFE_SDX/src/main/resources/scripts/"
clientsites:"TAMU:UH:UFL:UNF"
controllersite:"UFL (Gainesville, FL USA) XO Rack"
serversite:"RENCI (Chapel Hill, NC USA) XO Rack"
}
EOF

echo Copying SDX config to cc@$SDX_PUBLIC_IP
scp -q -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    $LOCAL_SDX_CONFIG \
    cc@${SDX_PUBLIC_IP}:${REMOTE_SDX_CONFIG}
echo Done

Copying SDX config to cc@129.114.108.17
Done


#### Start the Dataplane Slice

In [26]:
echo Start SDX dataplane Slice
ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${SDX_PUBLIC_IP} \
    sudo ${REMOTE_BIN_DIR}/SafeSdxSliceServer -c $REMOTE_SDX_CONFIG  
echo Slice Creation Done!

Start SDX dataplane Slice
Slice Creation Done!


#### Start the SDX Server

In [ ]:
echo Start the SDX Server
ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${SDX_PUBLIC_IP} \
    sudo ${REMOTE_BIN_DIR}/SafeSdxServer -c $REMOTE_SDX_CONFIG

Start the SDX Server
starting exoplex.sdx server
20:01:30 [main] INFO  exoplex.sdx.core.SdxManager - Carrier Slice server with Service API: START
Feb 26, 2019 8:01:46 PM org.glassfish.grizzly.http.server.NetworkListener start
INFO: Started listener bound to [0.0.0.0:8888]
Feb 26, 2019 8:01:46 PM org.glassfish.grizzly.http.server.HttpServer start
INFO: [HttpServer] Started.
Sdx server has started, listening on http://0.0.0.0:8888/


# Please Cleanup Your Resources!!!

#### Delete SDX Stack

In [ ]:
echo Deleting stack ${SDX_STACK_NAME}
openstack stack delete -y --wait ${SDX_STACK_NAME}
echo Stack ${SDX_STACK_NAME} deleted

#### Delete Dataplane Slice

In [ ]:
#need to install exogeni phython script in the container
#python delete ${DATAPLANE_SLICE_NAME}

#### Delete RIAK Stack

In [ ]:
echo Deleting stack ${RIAK_STACK_NAME}
openstack stack delete -y --wait ${RIAK_STACK_NAME}
echo Stack ${RIAK_STACK_NAME} deleted